In [ ]:
# pip install ultralytics pillow lxml scikit-learn
# !pip install wget 
# Import library
import os
import xml.etree.ElementTree as ET
from PIL import Image
import pandas as pd
from ultralytics import YOLO
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Validation of trained model weights
# Load the trained YOLOv8 model
model = YOLO('yolov8n_trained.pt')  # Make sure the path to your trained model file is correct

# Evaluate the model on the test data
results = model.val(data='data.yaml', split='test')  # Specify the test split
print(results)
# Print the evaluation results
print(f"mAP: {results.box.map}")
print(f"mAP@50: {results.box.map50}")
print(f"mAP@75: {results.box.map75}")
print(f"Precision: {results.box.mp}")
print(f"Recall: {results.box.mr}")
# print(f"IoU: {results.box.iou}")
# print(f"Accuracy: {results.box}")

In [ ]:
# Load the trained YOLOv8 model
model = YOLO('exp1_yolov8n_trained.pt')  # Make sure the path to your trained model file is correct

# Get predictions from the model on the test data
results = model.predict(source='datasets/test/images', save=False)  # Specify the test images folder

# Define paths to the image and label folders
image_folder = 'datasets/test/images'  # Replace with the actual path to your test images folder
label_folder = 'datasets/test/labels'  # Replace with the actual path to your test labels folder

# Get image names from the folder
image_names = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

# Initialize lists to store ground truth labels and predicted labels
y_true = []
y_pred = []

# Map labels from the folder and extract the first value before space as label
for image_name in image_names:
    label_file = os.path.join(label_folder, os.path.splitext(image_name)[0] + '.txt')
    if os.path.exists(label_file):
        with open(label_file, 'r') as f:
            label = f.read().strip().split()[0]  # Extract the first value before space as label
            y_true.append(int(label))  # Convert label to integer
    else:
        y_true.append(0)  # Default value if label file does not exist

for result in results:
    if result.boxes is not None:
        for pred in result.boxes.cls:  # Assuming result.boxes.cls gives the predicted classes
            y_pred.append(int(pred.item()))  # Convert tensor to integer
    else:
        y_pred.append(0)  # Default value if no prediction is made

# Ensure y_pred has the same length as y_true by padding with default values if necessary
while len(y_pred) < len(y_true):
    y_pred.append(0)

# Create a DataFrame with image names, actual labels, and predicted labels
output_df = pd.DataFrame({
    'Image Name': image_names,
    'Actual Label': y_true,
    'Predicted Label': y_pred
})

# Save the DataFrame to a CSV file
output_df.to_csv('output/custom_data_training.csv', index=False)

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix using seaborn heatmap with correct axis labels and title
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion  for face mask detection')
plt.savefig('output/confusion_matrix.png')
plt.show()

print("Output file and confusion matrix png saved successfully.")


# Export the trained model to ONNX format
model.export(format='onnx')
from ultralytics import YOLO
from PIL import Image

# Load the trained YOLOv8 model
model = YOLO('exp1_yolov8n_trained.onnx')  # Make sure the path to your model file is correct

# Function to perform inference on an image and save the predicted image
def perform_inference(image_path, save_path):
    # Load the image
    img = Image.open(image_path)
    
    # Perform inference
    results = model.predict(img)
    
    # Save the predicted image
    results[0].save(save_path)  # Access the first result and save the image with bounding boxes

    # Optionally, you can print the results
    predictions = results[0].boxes  # Access the bounding boxes
    print(predictions)

# Example usage
image_path = 'datasets\predict\maksssksksss0.png'  # Replace with the path to your image
save_path = 'output\predicted\maksssksksss0.jpg'  # Replace with the path where you want to save the predicted image
perform_inference(image_path, save_path)

Image.open("output\confusion_matrix.png")

In [ ]:
# Export the trained model to ONNX format
model.export(format='onnx')

In [ ]:
from ultralytics import YOLO
from PIL import Image

# Load the trained YOLOv8 model
model = YOLO('trained_weights/yolov8n_trained.onnx')  # Make sure the path to your model file is correct

# Function to perform inference on an image and save the predicted image
def perform_inference(image_path, save_path):
    # Load the image
    img = Image.open(image_path)
    
    # Perform inference
    results = model.predict(img)
    
    # Save the predicted image
    results[0].save(save_path)  # Access the first result and save the image with bounding boxes

    # Optionally, you can print the results
    predictions = results[0].boxes  # Access the bounding boxes
    print(predictions)

# Example usage
image_path = 'datasets\predict\maksssksksss0.png'  # Replace with the path to your image
save_path = 'output\predicted\maksssksksss0.jpg'  # Replace with the path where you want to save the predicted image
perform_inference(image_path, save_path)